In [1]:
import tensorflow as tf
from cpsrl.models.mean import ConstantMean, LinearMean
from cpsrl.models.covariance import EQ
from cpsrl.models.gp import VFEGP
from cpsrl.models.gp import VFEGPStack

In [2]:
# Check constant mean works
constant_mean = ConstantMean(trainable=True,
                             dtype=tf.float64)

ones = tf.ones(shape=(2, 3), dtype=tf.float64)
constant_mean(ones)

# Check linear mean works
linear_mean = LinearMean(input_dim=3,
                         trainable=True,
                         dtype=tf.float64)
linear_mean(ones)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0., 0.])>

In [3]:
# Check EQ covariance works
log_coeff = 0.
log_scales = [-1., -1.]

# Check EQ kernel works
eq = EQ(log_coeff=log_coeff,
        log_scales=log_scales,
        trainable=True,
        dtype=tf.float64)

x1 = tf.random.uniform(shape=(4, 2), dtype=tf.float64)
x2 = tf.random.uniform(shape=(3, 2), dtype=tf.float64)

eq(x1, x2)

<tf.Tensor: shape=(4, 3), dtype=float64, numpy=
array([[0.81652651, 0.97262145, 0.48594971],
       [0.51257423, 0.76978557, 0.29885077],
       [0.47086533, 0.5900562 , 0.42057098],
       [0.99128309, 0.91615067, 0.84609404]])>

In [4]:
# Check VFEGP works
dtype = tf.float64
trainable_mean = True
trainable_cov = True
trainable_noise = True
trainable_inducing = True
log_coeff = -1.
log_scales = [-1., -1., -1.]
log_noise = -4.

num_data = 10
state_dim = 2
action_dim = 1
num_ind = 10

x_train = tf.random.uniform(shape=(num_data, state_dim + action_dim), 
                            dtype=dtype)
y_train = tf.random.uniform(shape=(num_data, state_dim),
                            dtype=dtype)

mean = LinearMean(input_dim=(state_dim+action_dim),
                  trainable=trainable_mean,
                  dtype=dtype)

cov = EQ(log_coeff=log_coeff,
         log_scales=log_scales,
         trainable=trainable_cov,
         dtype=dtype)

vfe_gp = VFEGP(mean=mean,
               cov=cov,
               x_train=x_train,
               y_train=y_train[:, :1],
               x_ind=None,
               num_ind=num_ind,
               trainable_inducing=trainable_inducing,
               log_noise=log_noise,
               trainable_noise=trainable_noise,
               dtype=dtype) 

vfe_gp.free_energy()

<tf.Tensor: shape=(), dtype=float64, numpy=-475.08974998372787>

In [5]:
# Check VFEGP works
dtype = tf.float64
trainable_mean = True
trainable_cov = True
trainable_noise = True
trainable_inducing = True
log_coeff = -1.
log_scales = [-1., -1., -1.]
log_noise = -4.

num_data = 10
state_dim = 2
action_dim = 1
num_ind = 10

x_train = tf.random.uniform(shape=(num_data, state_dim + action_dim), 
                            dtype=dtype)
y_train = tf.random.uniform(shape=(num_data, state_dim),
                            dtype=dtype)

means = [LinearMean(input_dim=(state_dim+action_dim),
                    trainable=trainable_mean,
                    dtype=dtype) for i in range(state_dim)]

covs = [EQ(log_coeff=log_coeff,
           log_scales=log_scales,
           trainable=trainable_cov,
           dtype=dtype) for i in range(state_dim)]

vfe_gps = [VFEGP(mean=means[i],
                 cov=covs[i],
                 x_train=x_train,
                 y_train=y_train[:, i:i+1],
                 x_ind=None,
                 num_ind=num_ind,
                 trainable_inducing=trainable_inducing,
                 log_noise=log_noise,
                 trainable_noise=trainable_noise,
                 dtype=dtype) 
           for i in range(state_dim)]

vfe_gp_stack = VFEGPStack(vfe_gps=vfe_gps,
                          dtype=dtype)

vfe_gp_stack.free_energy()

<tf.Tensor: shape=(), dtype=float64, numpy=-1077.4101985623483>

In [6]:
num_steps = 100
optimizer = tf.keras.optimizers.Adam(1e-1)

for step in range(num_steps + 1):
    
    with tf.GradientTape() as tape:
        
        tape.watch(vfe_gp_stack.trainable_variables)

        free_energy = vfe_gp_stack.free_energy()
        loss = - free_energy
        print(loss)
        
    gradients = tape.gradient(loss, vfe_gp_stack.trainable_variables)
    optimizer.apply_gradients(zip(gradients, vfe_gp_stack.trainable_variables))

tf.Tensor(1077.4101985623483, shape=(), dtype=float64)
tf.Tensor(600.1568117214771, shape=(), dtype=float64)
tf.Tensor(350.9825796684705, shape=(), dtype=float64)
tf.Tensor(203.71584712750052, shape=(), dtype=float64)
tf.Tensor(125.17948869142504, shape=(), dtype=float64)
tf.Tensor(80.44765495187502, shape=(), dtype=float64)
tf.Tensor(52.69064621768966, shape=(), dtype=float64)
tf.Tensor(36.10546158279715, shape=(), dtype=float64)
tf.Tensor(26.274699675655615, shape=(), dtype=float64)
tf.Tensor(20.01083281509097, shape=(), dtype=float64)
tf.Tensor(15.347078177783004, shape=(), dtype=float64)
tf.Tensor(11.687677948779616, shape=(), dtype=float64)
tf.Tensor(9.16157144098695, shape=(), dtype=float64)
tf.Tensor(7.590682551332596, shape=(), dtype=float64)
tf.Tensor(6.5562252471330975, shape=(), dtype=float64)
tf.Tensor(5.846915304005812, shape=(), dtype=float64)
tf.Tensor(5.301024627424062, shape=(), dtype=float64)
tf.Tensor(4.732507002641701, shape=(), dtype=float64)
tf.Tensor(4.1929926040

In [24]:
num_features = 100
x = tf.reshape(tf.range(15, dtype=dtype), (5, 3))

post_sample = vfe_gp_stack.sample_posterior(num_features=num_features)

In [32]:
post_sample(x, add_noise=True)

<dtype: 'float64'> float64
<dtype: 'float64'> float64


<tf.Tensor: shape=(5, 2), dtype=float64, numpy=
array([[-0.02734332,  0.06631785],
       [ 0.0988031 ,  0.06427051],
       [-0.01539494,  0.07682769],
       [-0.13729333,  0.02333228],
       [ 0.02665974, -0.20138414]])>